# Welcome to ProgPy's Kalman Filter Example!

In this example demonstrates use of the Kalman Filter, a [State Estimator](https://nasa.github.io/progpy/prog_algs_guide.html#state-estimation),  with a [LinearModel](https://nasa.github.io/progpy/api_ref/prog_models/LinearModel.html).

The Kalman Filter (KF) is an optimal recursive computational algorithm designed to estimate the state of a linear dynamic system from noisy measurements. It is widely used in various applications, from robotics and aerospace to finance.

## How the Kalman Filter Works

1. **Initialization**: The filter starts with an initial state estimate and an initial estimate of the uncertainty.
2. **Measurement Update (Correction)**: Every time a new measurement is received, the KF performs an update to correct its current state estimate. 
    <font color='teal'>This correction is based on the difference between the predicted measurement and the actual received measurement, also known as the "residual". The amount of correction applied is determined by the Kalman gain, which itself is a function of the prediction and measurement uncertainties.</font>
2. **Prediction**: The filter predicts the next state and uncertainty based on the system's dynamics. Based on the corrected state estimate and the system dynamics, the KF predicts what the state will be at the next time step. <font color='teal'>This prediction also increases the uncertainty because, without new measurements, we're relying solely on our model, which might not capture all the complexities of the real world.</font>
3. **Update**: When a new measurement is available, the filter updates its state and uncertainty estimates based on the new data.


## Kalman Filter in the ProgPy Package

The Kalman Filter is designed to work with a `LinearModel`. The main components include:

- **Model**: A prognostics model used in state estimation.
- **Initial State**: Defined by the keys from `model.states`.
- **Noise Matrices**:
  - `Q`: Process Noise Matrix
  - `R`: Measurement Noise Matrix
- **Configuration Parameters**:
  - `alpha`: Scaling parameter for the filter.
  - `t0`: Starting time.
  - `dt`: Maximum timestep for prediction.

### Importing Modules

In [ ]:
from progpy.state_estimators import KalmanFilter
from progpy.models import LinearThrownObject
import matplotlib.pyplot as plt

First, let's instantiate the model.


In [ ]:
m = LinearThrownObject(process_noise = 0.1, measurement_noise = 0.1)

Next we'll instantiate the Kalman Filter State Estimator and we'll define the initial state to be slightly off of actual

In [ ]:
x_guess = m.StateContainer({'x': 5, 'v': 35}) # Guess of initial state
# Note: actual is {'x': 1.83, 'v': 40}
kf = KalmanFilter(m, x_guess)

Before we run our Kalman Filter, let's define some constants that we'll use for our simulation method and the kalman filter itself. We'll also be instantiating a few empty lists that we'll use for plotting later on.

In [ ]:
dt = 0.01  # Time step (s)
print_freq = 50  # Print every print_freq'th iteration
x = m.initialize()
u = m.InputContainer({})  # No input for this model

# Initialize lists for plotting
estimates_x = []
truths_x = []
diffs_x = []

estimates_v = []
truths_v = []
diffs_v = []

Here we're using simulated data from the thrown_object. In a real application you would be using sensor data from the system.

In [ ]:
for i in range(500):
    # Get simulated output (would be measured in a real application)
    z = m.output(x)

    # Estimate New State
    kf.estimate(i*dt, u, z)
    x_est = kf.x.mean

    # Print Results
    if i%print_freq == 0:  # Print every print_freq'th iteration
        print(f"t: {i*dt:.2f}\n\tEstimate: {x_est}\n\tTruth: {x}")
        diff = {key: abs(x_est[key] - x[key]) for key in x.keys()}
        print(f"\t Diff: {diff}")
        
        # Append values for plotting
        estimates_x.append(x_est['x'])
        truths_x.append(x['x'])
        diffs_x.append(diff['x'])

        estimates_v.append(x_est['v'])
        truths_v.append(x['v'])
        diffs_v.append(diff['v'])

    # Update Real state for next step
    x = m.next_state(x, u, dt)

Let's plot the results to visualize the difference! First, let's see the `x` values!

In [ ]:
times = [i*dt for i in range(0, 500, print_freq)]

# Plotting for 'x'
plt.figure(figsize=(10, 5))
plt.plot(times, estimates_x, label="Estimate x", color='blue')
plt.plot(times, truths_x, label="Truth x", color='green')
plt.plot(times, diffs_x, label="Difference x", color='red', linestyle='--')
plt.xlabel("Time")
plt.ylabel("x Value")
plt.legend()
plt.title("Comparison of Estimate, Truth, and Difference for x")
plt.grid(True)
plt.show()

Now, let's take a look at the `v` values!

In [ ]:
# Plotting for 'v'
plt.figure(figsize=(10, 5))
plt.plot(times, estimates_v, label="Estimate v", color='blue')
plt.plot(times, truths_v, label="Truth v", color='green')
plt.plot(times, diffs_v, label="Difference v", color='red', linestyle='--')
plt.xlabel("Time")
plt.ylabel("v Value")
plt.legend()
plt.title("Comparison of Estimate, Truth, and Difference for v")
plt.grid(True)
plt.show()

In our simulation, the Kalman Filter is applied iteratively. At each time step, the filter predicts the new state, receives a measurement, updates its estimate, and then moves to the next time step. As observed, the difference between the estimated state and the true state decreases over time, indicating the filter's convergence.

### Conclusion

The Kalman Filter is a powerful tool for state estimation in noisy environments. Given an initial state estimate and associated uncertainty, the filter processes measurements to continuously update its beliefs about the system's state. As more measurements are processed, the filter's state estimates become more accurate, often converging to the true state of the system. This behavior is evident in the provided code and simulation results.

For more information on, please view our ProgPy [Documentation](https://nasa.github.io/progpy/index.html).